# TREC-50
# ESN Bayesian Optimization (``ax-platform``) + Simulations

## Librairies

In [ ]:
# !pip install transformers==4.8.2
# !pip install datasets==1.7.0
# !pip install ax-platform==0.1.20

In [1]:
import os
import sys
sys.path.insert(0, os.path.abspath("../..")) # comment this if library is pip installed

In [2]:
import io
import re
import pickle
from timeit import default_timer as timer

from tqdm.notebook import tqdm

import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim

from datasets import load_dataset, Dataset, concatenate_datasets
from transformers import AutoTokenizer
from transformers import BertModel
from transformers.data.data_collator import DataCollatorWithPadding

from ax import optimize
from ax.plot.contour import plot_contour
from ax.plot.trace import optimization_trace_single_method
from ax.service.managed_loop import optimize
from ax.utils.notebook.plotting import render, init_notebook_plotting

import esntorch.core.reservoir as res
import esntorch.core.learning_algo as la
import esntorch.core.merging_strategy as ms
import esntorch.core.esn as esn

In [3]:
%config Completer.use_jedi = False
%load_ext autoreload
%autoreload 2

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [5]:
SEED = 42

## Global variables

In [6]:
pooling_strategy = 'mean' # 'last' or mean'

In [7]:
RESULTS_PATH = 'Results/ESN_v2' # put your path here
CACHE_DIR = '/huggingface_datasets' # put your path here
PARAMS_FILE = 'trec-50_params_uniform_'+pooling_strategy+'-pooling.pkl'
RESULTS_FILE = 'trec-50_results_esn_uniform_'+pooling_strategy+'-pooling_FINAL.pkl'

In [8]:
RESULTS_FILE

'trec-50_results_esn_uniform_mean-pooling_FINAL.pkl'

## Dataset

In [9]:
# rename correct column as 'labels': depends on the dataset you load

def tokenize(sample):
    """Tokenize sample"""
    
    sample = tokenizer(sample['text'], truncation=True, padding=False, return_length=True)
    
    return sample
    
def load_and_enrich_dataset(dataset_name, split, cache_dir):
    """
    Load dataset from the datasets library of HuggingFace.
    Tokenize and add length.
    """
    
    # Load dataset
    dataset = load_dataset(dataset_name, split=split, cache_dir=CACHE_DIR)
    
    # Rename label column for tokenization purposes (use 'label-fine' for fine-grained labels)
    dataset = dataset.rename_column('label-fine', 'labels')
    
    # Tokenize data
    dataset = dataset.map(tokenize, batched=True)
    dataset = dataset.rename_column('length', 'lengths')
    dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels', 'lengths'])
    
    return dataset

In [ ]:
# tokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

# train set
full_train_dataset = load_and_enrich_dataset('trec', split='train', cache_dir=CACHE_DIR).sort("lengths")

# mini train and val sets: 70%/30% of 30%
# XXX here we increase the train_size split since too many labels
train_val_datasets = full_train_dataset.train_test_split(train_size=0.99, shuffle=True)
train_val_datasets = train_val_datasets['train'].train_test_split(train_size=0.7, shuffle=True)
train_dataset = train_val_datasets['train'].sort("lengths")
val_dataset = train_val_datasets['test'].sort("lengths")

# test set
test_dataset = load_and_enrich_dataset('trec', split='test', cache_dir=CACHE_DIR).sort("lengths")

dataset_d = {
    'full_train': full_train_dataset,
    'train': train_dataset,
    'val': val_dataset,
    'test': test_dataset
    }

# dataloaders
dataloader_d = {}
for k, v in dataset_d.items():
    dataloader_d[k] = torch.utils.data.DataLoader(v, batch_size=256, collate_fn=DataCollatorWithPadding(tokenizer))

In [11]:
dataset_d

{'full_train': Dataset({
     features: ['attention_mask', 'input_ids', 'label-coarse', 'labels', 'lengths', 'text', 'token_type_ids'],
     num_rows: 5452
 }),
 'train': Dataset({
     features: ['attention_mask', 'input_ids', 'label-coarse', 'labels', 'lengths', 'text', 'token_type_ids'],
     num_rows: 3777
 }),
 'val': Dataset({
     features: ['attention_mask', 'input_ids', 'label-coarse', 'labels', 'lengths', 'text', 'token_type_ids'],
     num_rows: 1620
 }),
 'test': Dataset({
     features: ['attention_mask', 'input_ids', 'label-coarse', 'labels', 'lengths', 'text', 'token_type_ids'],
     num_rows: 500
 })}

In [12]:
# set(dataset_d['train']['labels'].tolist())
# set(dataset_d['val']['labels'].tolist())

## Optimization

- Parameters

In [13]:
ESN_PARAMS = {
            'embedding': 'bert-base-uncased', # TEXT.vocab.vectors,
            'distribution' : 'uniform',              # uniform, gaussian
            'input_dim' : 768,                        # dim of encoding!
            'reservoir_dim' : None,
            'bias_scaling' : 0.0,
            'sparsity' : None,
            'spectral_radius' : None,
            'leaking_rate': None,
            'activation_function' : 'tanh',
            'input_scaling' : None,
            'mean' : 0.0,
            'std' : 1.0,
            'learning_algo' : None,
            'criterion' : None,
            'optimizer' : None,
            'merging_strategy' : pooling_strategy,
            'lexicon' : None,
            'bidirectional' : False, # False
            'mode' : None,
            'device' : device,
            'seed' : None
             }

In [14]:
def plug_parameters(args_d):
    
    params_d = ESN_PARAMS.copy()
    
    for k, v in args_d.items():
        if k in params_d.keys():
            params_d[k] = v
            
    return params_d

In [15]:
# args_d = {'reservoir_dim': 1000, 
#      'spectral_radius': 0.9, 
#      'leaking_rate': 0.5, 
#      'input_scaling': 0.1, 
#      'sparsity': 0.99, 
#      'activation_function': 'tanh', 
#      'alpha': 1.0}

# d = plug_parameters(args_d)

# d

- Fitness function

In [16]:
def warm_up(ESN, dataset_d):
    """Warm up the ESN."""
    
    nb_sentences = 10
    
    for i in range(nb_sentences): 

        sentence = dataset_d["train"].select([i])
        dataloader_tmp = torch.utils.data.DataLoader(sentence, 
                                                     batch_size=1, 
                                                     collate_fn=DataCollatorWithPadding(tokenizer))  

        for sentence in dataloader_tmp:
            
            ESN.warm_up(sentence)

In [17]:
def fitness(reservoir_dim=1000,
            spectral_radius=0.9,
            leaking_rate=0.5,
            input_scaling=0.1, 
            sparsity=0.99,
            activation_function='tanh',
            alpha=1.0,
            dataset_d=dataset_d, 
            dataloader_d=dataloader_d, 
            seed_l=[1234321, 420, 888],#, 1979, 7], # 5 seeds
            return_test_acc=False):
    
    # XXX
    if return_test_acc:
        seed_l = [1654547824, 1947, 1949, 1979, 1083, 1234321, 7, 42, 888, 5998989775] # 10 seeds
    elif not return_test_acc:
        seed_l = [1234321, 420, 888] # 3 seeds
    print('SEEDS:', seed_l)
    # XXX
    
    # create esn parameters
    args_d = locals()
    esn_params = plug_parameters(args_d)
        
    # BO mode: param optimization (esn_params['mode'] = 'esn')
    if not return_test_acc:
    
        for seed in tqdm(seed_l):
            
            acc_l = []
            
            # model
            esn_params['mode'] = 'esn'
            esn_params['seed'] = seed
            ESN = esn.EchoStateNetwork(**esn_params)
            ESN.learning_algo = la.RidgeRegression(alpha = alpha)
            ESN = ESN.to(device)

            # warm up
            warm_up(ESN, dataset_d)
            
            # train
            LOSS = ESN.fit(dataloader_d["train"])                           # mini train set
            acc = ESN.predict(dataloader_d["val"], verbose=False)[1].item() # mini val set
            acc_l.append(acc)
            
            # clean objects
            del ESN.learning_algo
            del ESN.criterion
            del ESN.merging_strategy
            del ESN
            torch.cuda.empty_cache()
            
        # results
        return np.mean(acc_l) 
          
    # prediction mode: once params have been optimized
    elif return_test_acc:
        
        # esn
        acc_esn_l = []
        time_esn_l = []
        # custom baseline
        acc_bs_l = []
        time_bs_l = []
        
        for seed in tqdm(seed_l):
            
            for mode in tqdm(['esn', 'linear_layer']):
            
                # model
                esn_params['mode'] = mode
                esn_params['seed'] = seed
                ESN = esn.EchoStateNetwork(**esn_params)
                ESN.learning_algo = la.RidgeRegression(alpha = alpha)
                ESN = ESN.to(device)
                print('RES DIM:', ESN.reservoir.reservoir_dim)

                # warm up
                warm_up(ESN, dataset_d)

                # train
                t0 = timer()
                LOSS = ESN.fit(dataloader_d["full_train"])                       # full train set
                t1 = timer()
                acc = ESN.predict(dataloader_d["test"], verbose=False)[1].item() # full test set
                
                # clean objects
                del ESN.learning_algo
                del ESN.criterion
                del ESN.merging_strategy
                del ESN
                torch.cuda.empty_cache()
                
                # results
                if mode == 'esn':
                    time_esn_l.append(t1 - t0)
                    acc_esn_l.append(acc)
                elif mode == 'linear_layer':
                    time_bs_l.append(t1 - t0)
                    acc_bs_l.append(acc)
            
        results_d = {}
        results_d['esn'] = [np.mean(acc_esn_l), np.std(acc_esn_l), np.mean(time_esn_l), np.std(time_esn_l)]
        results_d['linear_layer'] = [np.mean(acc_bs_l), np.std(acc_bs_l), np.mean(time_bs_l), np.std(time_bs_l)]
            
        return results_d

In [18]:
# %%time

# fitness(
#         reservoir_dim=1000,
#         spectral_radius=0.9,
#         leaking_rate=0.5,
#         input_scaling=0.1, 
#         sparsity=0.99,
#         activation_function='tanh',
#         alpha=1.0,
#         dataset_d=dataset_d, 
#         dataloader_d=dataloader_d,
#         seed_l=[1234321, 7],
#         return_test_acc=True
# )

In [19]:
def wrapped_fitness(d, return_test_acc=False):
    
    return fitness(reservoir_dim=d['reservoir_dim'], # will be in the loop 
                   spectral_radius=d['spectral_radius'],
                   leaking_rate=d['leaking_rate'],
                   input_scaling=d['input_scaling'],
                   sparsity=d['sparsity'],
                   activation_function=d['activation_function'],
                   alpha=d['alpha'],
                   dataset_d=dataset_d,
                   dataloader_d=dataloader_d,
                   return_test_acc=return_test_acc)

- Optimization

In [19]:
best_params_d = {}

for res_dim in tqdm([500, 1000, 3000, 5000]):
    
    best_parameters, best_values, experiment, model = optimize(
            parameters=[
              {
                "name": "reservoir_dim",
                "value_type": "int",
                "type": "fixed",
                "value": res_dim,
              },
              {
                "name": "activation_function",
                "value_type": "str",
                "type": "fixed",
                "value": 'tanh',
              },
              {
                "name": "spectral_radius",
                "value_type": "float",
                "type": "choice",
                "values": [0.5, 1.0, 1.5],
              },
              {
                "name": "leaking_rate",
                "value_type": "float",
                "type": "choice",
                "values": [0.1, 0.5, 0.9],
              },
              {
                "name": "input_scaling",
                "value_type": "float",
                "type": "choice",
                "values": [0.1, 1.0],
              },
              {
                "name": "sparsity",
                "value_type": "float",
                "type": "choice",
                "values": [0.0, 0.9],
              },
              {
                "name": "alpha",
                "value_type": "float",
                "type": "choice",
                "values": [0.1, 1.0, 10.0, 100.0],
              }
            ],
            # Booth function
            evaluation_function = wrapped_fitness,
            minimize = False,
            objective_name = 'val_accuracy',
            total_trials = 30 # nb iterations here
        )
    
    # results
    best_params_d[res_dim] = {}
    best_params_d[res_dim]['best_parameters'] = best_parameters
    best_params_d[res_dim]['best_values'] = best_values
    best_params_d[res_dim]['experiment'] = experiment
    # best_params_d[res_dim]['model'] = model

  0%|          | 0/4 [00:00<?, ?it/s]

[INFO 01-18 09:06:54] ax.modelbridge.dispatch_utils: Using Sobol generation strategy.
[INFO 01-18 09:06:54] ax.service.managed_loop: Started full optimization with 30 steps.
[INFO 01-18 09:06:54] ax.service.managed_loop: Running optimization trial 1...


  0%|          | 0/3 [00:00<?, ?it/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Model downloaded: bert-base-uncased


...

Model downloaded: bert-base-uncased


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Model downloaded: bert-base-uncased


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Model downloaded: bert-base-uncased


In [20]:
best_params_d

{500: {'best_parameters': {'spectral_radius': 0.5,
   'leaking_rate': 0.9,
   'input_scaling': 0.1,
   'sparsity': 0.0,
   'alpha': 1.0,
   'reservoir_dim': 500,
   'activation_function': 'tanh'},
  'best_values': ({'val_accuracy': 67.28395080566406},
   {'val_accuracy': {'val_accuracy': nan}}),
  'experiment': SimpleExperiment(None)},
 1000: {'best_parameters': {'spectral_radius': 0.5,
   'leaking_rate': 0.9,
   'input_scaling': 0.1,
   'sparsity': 0.0,
   'alpha': 1.0,
   'reservoir_dim': 1000,
   'activation_function': 'tanh'},
  'best_values': ({'val_accuracy': 70.43209838867188},
   {'val_accuracy': {'val_accuracy': nan}}),
  'experiment': SimpleExperiment(None)},
 3000: {'best_parameters': {'spectral_radius': 0.5,
   'leaking_rate': 0.9,
   'input_scaling': 0.1,
   'sparsity': 0.0,
   'alpha': 10.0,
   'reservoir_dim': 3000,
   'activation_function': 'tanh'},
  'best_values': ({'val_accuracy': 75.6172866821289},
   {'val_accuracy': {'val_accuracy': nan}}),
  'experiment': SimpleE

In [21]:
# save parameters

with open(os.path.join(RESULTS_PATH, PARAMS_FILE), 'wb') as fh:
    pickle.dump(best_params_d, fh)

In [21]:
# load results
with open(os.path.join(RESULTS_PATH, PARAMS_FILE), 'rb') as fh:
    best_params_d = pickle.load(fh)

In [22]:
PARAMS_FILE

'trec-50_params_uniform_mean-pooling.pkl'

In [23]:
best_params_d

{500: {'best_parameters': {'spectral_radius': 1.0,
   'leaking_rate': 0.1,
   'input_scaling': 0.1,
   'sparsity': 0.9,
   'alpha': 1.0,
   'reservoir_dim': 500,
   'activation_function': 'tanh'},
  'best_values': ({'val_accuracy': 72.90123748779297},
   {'val_accuracy': {'val_accuracy': nan}}),
  'experiment': SimpleExperiment(None)},
 1000: {'best_parameters': {'spectral_radius': 0.5,
   'leaking_rate': 0.1,
   'input_scaling': 0.1,
   'sparsity': 0.0,
   'alpha': 0.1,
   'reservoir_dim': 1000,
   'activation_function': 'tanh'},
  'best_values': ({'val_accuracy': 76.72840118408203},
   {'val_accuracy': {'val_accuracy': nan}}),
  'experiment': SimpleExperiment(None)},
 3000: {'best_parameters': {'spectral_radius': 1.0,
   'leaking_rate': 0.1,
   'input_scaling': 0.1,
   'sparsity': 0.9,
   'alpha': 1.0,
   'reservoir_dim': 3000,
   'activation_function': 'tanh'},
  'best_values': ({'val_accuracy': 81.48148345947266},
   {'val_accuracy': {'val_accuracy': nan}}),
  'experiment': SimpleE

## Simulations
- Using best paramameters

In [24]:
# results

results_d = {}

for res_dim in tqdm([500, 1000, 3000, 5000]):
    
    best_parameters = best_params_d[res_dim]['best_parameters']
    d = wrapped_fitness(best_parameters, return_test_acc=True)
    results_d[res_dim] = d
    print("Experiment finished.")

  0%|          | 0/4 [00:00<?, ?it/s]

SEEDS: [1654547824, 1947, 1949, 1979, 1083, 1234321, 7, 42, 888, 5998989775]


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


...

  0%|          | 0/2 [00:00<?, ?it/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Model downloaded: bert-base-uncased
RES DIM: 5000


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Model downloaded: bert-base-uncased
RES DIM: 5000
Experiment finished.


In [25]:
results_d

{500: {'esn': [74.1800033569336,
   0.8919638237605361,
   4.089984841272235,
   0.20947851286539593],
  'linear_layer': [74.4400032043457,
   1.0873826209559703,
   4.27845759075135,
   0.7351729010297224]},
 1000: {'esn': [79.98000411987304,
   0.5688588119244873,
   4.293796676490456,
   0.5757323035634171],
  'linear_layer': [79.28000335693359,
   0.6939740277394423,
   4.279738957341761,
   0.3109984943454546]},
 3000: {'esn': [84.82000427246093,
   0.554616445717667,
   5.331064376421272,
   0.3170257213044556],
  'linear_layer': [82.72000503540039,
   0.7166589232046979,
   5.341141401138157,
   0.13241709337689545]},
 5000: {'esn': [86.48000411987304,
   0.7547193073672976,
   9.810478774737566,
   0.2658847292288352],
  'linear_layer': [81.60000457763672,
   5.531726564039616,
   11.306728410441428,
   0.941272693915772]}}

In [26]:
RESULTS_FILE

'trec-50_results_esn_uniform_mean-pooling_FINAL.pkl'

In [27]:
# save results

with open(os.path.join(RESULTS_PATH, RESULTS_FILE), 'wb') as fh:
    pickle.dump(results_d, fh)

In [26]:
# # load results
# with open(os.path.join(RESULTS_PATH, RESULTS_FILE), 'rb') as fh:
#     results_d = pickle.load(fh)

In [27]:
# results_d